In [0]:
# run to install needed modules
!pip install geopandas

     |████████████████████████████████| 921kB 3.3MB/s 
     |████████████████████████████████| 10.1MB 15.2MB/s 
     |████████████████████████████████| 11.8MB 258kB/s 


In [0]:
# run to mount my google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

# note might need to change this if the folder gets moved
import os
os.chdir('My Drive/In-progress/PacRISA_landuse/Scripts')

In [0]:
# import modules
import os
import numpy as np
import pandas as pd
import gdal
import ogr
import geopandas as gpd
import glob
import shutil
import fiona

In [0]:
def createBuffer(inputfn, outputBufferfn, bufferDist):
    inputds = ogr.Open(inputfn)
    inputlyr = inputds.GetLayer()

    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputBufferfn):
        shpdriver.DeleteDataSource(outputBufferfn)
    outputBufferds = shpdriver.CreateDataSource(outputBufferfn)
    bufferlyr = outputBufferds.CreateLayer(outputBufferfn, geom_type=ogr.wkbPolygon)
    featureDefn = bufferlyr.GetLayerDefn()

    for feature in inputlyr:
        ingeom = feature.GetGeometryRef()
        geomBuffer = ingeom.Buffer(bufferDist)

        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(geomBuffer)
        bufferlyr.CreateFeature(outFeature)
        outFeature = None

In [0]:
# set/create GENeral workspace
workspace = os.path.join('.', 'workspace')
if not os.path.exists(workspace):
    os.makedirs(workspace)

# Paths to data
GIS_FOLDER = os.path.join('..', 'Raw_GIS_Data')

# path to the grid bound
Grid_shp = os.path.join(GIS_FOLDER, 'grid_bound.shp')

if not os.path.exists(os.path.join('.', 'output//')):
    os.makedirs(os.path.join('.', 'output//'))
                        
if not os.path.exists(os.path.join('.', 'output//Figures//')):
    os.makedirs(os.path.join('.', 'output//Figures//'))
fig_path =  (os.path.join('.', 'output//Figures//'))

# define where the land is too steep to build
steep_land = os.path.join(GIS_FOLDER, 'Slopes', 'steep_land_25deg.shp')

In [0]:
# path to the raw land use data set
LU_shp = os.path.join(GIS_FOLDER, 'Land_use', 'LU_BIG_simp3m.shp')

In [0]:
# Original

type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(LU_shp)

for idx, lutype in enumerate(type_list):
    dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]
    dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))

    # use OGR to create the buffer file 
    print("buffering {}".format(lutype))
    %time  createBuffer(os.path.join(workspace, 'tempselect.shp'), os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)), Buf_dist_list[idx])
    



buffering Agroforest
CPU times: user 24.8 s, sys: 408 ms, total: 25.2 s
Wall time: 25.4 s
buffering Cultivated_Land
CPU times: user 40.2 s, sys: 163 ms, total: 40.4 s
Wall time: 40.6 s
buffering Developed_Woodlands
CPU times: user 1min 14s, sys: 456 ms, total: 1min 15s
Wall time: 1min 15s
buffering Open_Space
CPU times: user 3min, sys: 485 ms, total: 3min 1s
Wall time: 3min 1s
buffering Impervious_Surfaces
CPU times: user 1min 25s, sys: 96.8 ms, total: 1min 25s
Wall time: 1min 25s
buffering Buildings
CPU times: user 1min 11s, sys: 84.8 ms, total: 1min 11s
Wall time: 1min 11s


In [0]:
# at 25 m simplification

copy_LU_shp = os.path.join(GIS_FOLDER,'Land_use', 'secretstash_can deletenow',  "LU_test_file_small_simp.shp")


type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(copy_LU_shp)

for idx, lutype in enumerate(type_list):
    dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]
    dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))

    # use OGR to create the buffer file 
    print("buffering {}".format(lutype))
    %time  createBuffer(os.path.join(workspace, 'tempselect.shp'), os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)), Buf_dist_list[idx])
    
    print("erasing {}".format(lutype))
    Erasein1 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Buildings')))
    Erasein2 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Impervious_Surfaces')))

    %time erasestep1time = gpd.overlay(Erasein1, Erasein2, how='difference')
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong1')))

    print("unioning back in on copymaster {}".format('1'))
    %time erasestep2time = gpd.overlay(Erasein1, Erasein2, how='union')   
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong2')))




buffering Agroforest
Wall time: 24.9 ms
erasing Agroforest
Wall time: 374 ms
unioning back in on copymaster 1
Wall time: 701 ms
buffering Cultivated_Land
Wall time: 21.9 ms
erasing Cultivated_Land
Wall time: 393 ms
unioning back in on copymaster 1
Wall time: 672 ms
buffering Developed_Woodlands
Wall time: 62.5 ms
erasing Developed_Woodlands
Wall time: 393 ms
unioning back in on copymaster 1
Wall time: 672 ms
buffering Open_Space
Wall time: 126 ms
erasing Open_Space
Wall time: 382 ms
unioning back in on copymaster 1
Wall time: 672 ms
buffering Impervious_Surfaces
Wall time: 110 ms
erasing Impervious_Surfaces
Wall time: 378 ms
unioning back in on copymaster 1
Wall time: 686 ms
buffering Buildings
Wall time: 156 ms
erasing Buildings
Wall time: 365 ms
unioning back in on copymaster 1
Wall time: 656 ms


In [0]:
# at 10 m simplification

copy_LU_shp = os.path.join(GIS_FOLDER,'Land_use', 'secretstash_can deletenow',  "LU_test_file_small_simp10.shp")


type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(copy_LU_shp)

for idx, lutype in enumerate(type_list):
    dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]
    dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))

    # use OGR to create the buffer file 
    print("buffering {}".format(lutype))
    %time  createBuffer(os.path.join(workspace, 'tempselect.shp'), os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)), Buf_dist_list[idx])
    
    print("erasing {}".format(lutype))
    Erasein1 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Buildings')))
    Erasein2 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Impervious_Surfaces')))

    %time erasestep1time = gpd.overlay(Erasein1, Erasein2, how='difference')
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong1')))

    print("unioning back in on copymaster {}".format('1'))
    %time erasestep2time = gpd.overlay(Erasein1, Erasein2, how='union')   
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong2')))

buffering Agroforest
Wall time: 18.9 ms
erasing Agroforest
Wall time: 384 ms
unioning back in on copymaster 1
Wall time: 677 ms
buffering Cultivated_Land
Wall time: 23.9 ms
erasing Cultivated_Land
Wall time: 403 ms
unioning back in on copymaster 1
Wall time: 720 ms
buffering Developed_Woodlands
Wall time: 70.8 ms
erasing Developed_Woodlands
Wall time: 431 ms
unioning back in on copymaster 1
Wall time: 708 ms
buffering Open_Space
Wall time: 125 ms
erasing Open_Space
Wall time: 384 ms
unioning back in on copymaster 1
Wall time: 712 ms
buffering Impervious_Surfaces
Wall time: 114 ms
erasing Impervious_Surfaces
Wall time: 421 ms
unioning back in on copymaster 1
Wall time: 727 ms
buffering Buildings
Wall time: 145 ms
erasing Buildings
Wall time: 387 ms
unioning back in on copymaster 1
Wall time: 729 ms


In [0]:
# at 2 m simplification

copy_LU_shp = os.path.join(GIS_FOLDER,'Land_use', 'secretstash_can deletenow',  "LU_test_file_small_simp2.shp")


type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(copy_LU_shp)

for idx, lutype in enumerate(type_list):
    dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]
    dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))

    # use OGR to create the buffer file 
    print("buffering {}".format(lutype))
    %time  createBuffer(os.path.join(workspace, 'tempselect.shp'), os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)), Buf_dist_list[idx])
    
    print("erasing {}".format(lutype))
    Erasein1 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Buildings')))
    Erasein2 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Impervious_Surfaces')))

    %time erasestep1time = gpd.overlay(Erasein1, Erasein2, how='difference')
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong1')))

    print("unioning back in on copymaster {}".format('1'))
    %time erasestep2time = gpd.overlay(Erasein1, Erasein2, how='union')   
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong2')))

buffering Agroforest
Wall time: 33.9 ms
erasing Agroforest
Wall time: 397 ms
unioning back in on copymaster 1
Wall time: 706 ms
buffering Cultivated_Land
Wall time: 42.9 ms
erasing Cultivated_Land
Wall time: 396 ms
unioning back in on copymaster 1
Wall time: 723 ms
buffering Developed_Woodlands
Wall time: 117 ms
erasing Developed_Woodlands
Wall time: 389 ms
unioning back in on copymaster 1
Wall time: 769 ms
buffering Open_Space
Wall time: 173 ms
erasing Open_Space
Wall time: 376 ms
unioning back in on copymaster 1
Wall time: 657 ms
buffering Impervious_Surfaces
Wall time: 167 ms
erasing Impervious_Surfaces
Wall time: 500 ms
unioning back in on copymaster 1
Wall time: 926 ms
buffering Buildings
Wall time: 184 ms
erasing Buildings
Wall time: 518 ms
unioning back in on copymaster 1
Wall time: 859 ms


In [0]:
# Point at BIG land use dataset at 2 m simplification

copy_LU_shp = os.path.join(GIS_FOLDER,'Land_use', 'secretstash_can deletenow',  "LU_BIG_simp2.shp")


type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(LU_shp)

for idx, lutype in enumerate(type_list):
    dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]
    dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))

    # use OGR to create the buffer file 
    print("buffering {}".format(lutype))
    %time  createBuffer(os.path.join(workspace, 'tempselect.shp'), os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)), Buf_dist_list[idx])
    
    print("erasing {}".format(lutype))
    Erasein1 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Buildings')))
    Erasein2 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Impervious_Surfaces')))

    %time erasestep1time = gpd.overlay(Erasein1, Erasein2, how='difference')
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong1')))

    print("unioning back in on copymaster {}".format('1'))
    %time erasestep2time = gpd.overlay(Erasein1, Erasein2, how='union')   
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong2')))

buffering Agroforest
CPU times: user 26.9 s, sys: 151 ms, total: 27.1 s
Wall time: 27.2 s
erasing Agroforest


In [0]:
# Point at BIG land use dataset at 3 m simplification

copy_LU_shp = os.path.join(GIS_FOLDER,'Land_use', 'secretstash_can deletenow',  "LU_BIG_simp3.shp")


type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(copy_LU_shp)

for idx, lutype in enumerate(type_list):
    dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]                # select and write desired land use type to its own file using Geopandas 
    dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))

    # use OGR to create the buffer file 
    print("buffering {}".format(lutype))
    %time  createBuffer(os.path.join(workspace, 'tempselect.shp'), os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)), Buf_dist_list[idx])
    
    print("erasing {}".format(lutype))
    Erasein1 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Buildings')))
    Erasein2 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Impervious_Surfaces')))

    %time erasestep1time = gpd.overlay(Erasein1, Erasein2, how='difference')
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong1')))

    print("unioning back in on copymaster {}".format('1'))
    %time erasestep2time = gpd.overlay(Erasein1, Erasein2, how='union')   
    erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong2')))

buffering Agroforest
Wall time: 4min 30s
erasing Agroforest


In [0]:
# documenting the results of the tests
copy_LU_shp = os.path.join(GIS_FOLDER,'Land_use', 'secretstash_can deletenow',  "LU_BIG_simp3.shp")


type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

%time dataSrc = gpd.read_file(copy_LU_shp)                         # Wall time: 2.9 s

# Point at BIG land use dataset at 3 m TIMING TEST 
dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]
%time dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))   # Wall time: 3.5 s   - Wall time: 8.41 s   - Wall time: 18 s

Wall time: 2.98 s
Wall time: 9.63 s


In [0]:

%time dataSrc = gpd.read_file(copy_LU_shp)                         # Wall time: 2.9 s

Wall time: 2.87 s


In [0]:
### Geopandas buffer test   12 min 53 sec 1st try  
type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]
idx = 0
lutype = 'Agroforest'

# use Gepandas  to create the buffer file     (OGR was running around 4 minutes for this, thus it this is comporable then its better because dont have to convert back from and to shapefiles? )
%time dataSrc_select['geometry'] = dataSrc_select.geometry.buffer(5)   # use geopandas instead of (OGR = Stupid_)

Wall time: 2min 50s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# NEW MWTHOD

In [0]:
copy_LU_shp = os.path.join(GIS_FOLDER,'Land_use', 'secretstash_can deletenow',  "LU_BIG_simp3.shp")

dataSrc = gpd.read_file(copy_LU_shp)  # Read in lad use data

# Step 1 buffer files 
Agroforest_select = dataSrc[dataSrc['LBLCLASS'] == 'Agroforest'].copy()
Agroforest_select['geometry'] = Agroforest_select.geometry.buffer(5)

Cultivated_Land_select = dataSrc[dataSrc['LBLCLASS'] == 'Cultivated_Land'].copy()
Cultivated_Land_select['geometry'] = Cultivated_Land_select.geometry.buffer(4)

Developed_Woodlands_select = dataSrc[dataSrc['LBLCLASS'] == 'Developed_Woodlands'].copy()
Developed_Woodlands_select['geometry'] = Developed_Woodlands_select.geometry.buffer(3)

Open_Space_select = dataSrc[dataSrc['LBLCLASS'] == 'Open_Space'].copy()
Open_Space_select['geometry'] = Open_Space_select.geometry.buffer(1)

Impervious_Surfaces_select = dataSrc[dataSrc['LBLCLASS'] == 'Impervious_Surfaces'].copy()
Impervious_Surfaces_select['geometry'] = Impervious_Surfaces_select.geometry.buffer(0.2)

Buildings_select = dataSrc[dataSrc['LBLCLASS'] == 'Buildings'].copy()
Buildings_select['geometry'] = Buildings_select.geometry.buffer(0.5)


# Step 2 Create erase unions 
LU_frame_expanded = dataSrc.copy()   # copy the dataset 

%time erase_step1 = gpd.overlay(LU_frame_expanded, Agroforest_select, how='difference')           # difference them both  (listed as inner then outer)
%time LU_frame_expanded = gpd.GeoDataFrame( pd.concat( [Agroforest_select, erase_step1], ignore_index=True) )    # Keep the inner one 

%time erase_step2 = gpd.overlay(LU_frame_expanded, Cultivated_Land_select, how='difference')           # difference them both  (listed as inner then outer)
%time LU_frame_expanded = gpd.GeoDataFrame( pd.concat( [Cultivated_Land_select, erase_step2], ignore_index=True) )    # Keep the inner one 

%time erase_step3 = gpd.overlay(LU_frame_expanded, Developed_Woodlands_select, how='difference')           # difference them both  (listed as inner then outer)
%time LU_frame_expanded = gpd.GeoDataFrame( pd.concat( [Developed_Woodlands_select, erase_step3], ignore_index=True) )    # Keep the inner one 

%time erase_step4 = gpd.overlay(LU_frame_expanded, Open_Space_select, how='difference')           # difference them both  (listed as inner then outer)
%time LU_frame_expanded = gpd.GeoDataFrame( pd.concat( [Open_Space_select, erase_step4], ignore_index=True) )    # Keep the inner one 

%time erase_step5 = gpd.overlay(LU_frame_expanded, Impervious_Surfaces_select, how='difference')           # difference them both  (listed as inner then outer)
%time LU_frame_expanded = gpd.GeoDataFrame( pd.concat( [Impervious_Surfaces_select, erase_step5], ignore_index=True) )    # Keep the inner one 

%time erase_step6 = gpd.overlay(LU_frame_expanded, Buildings_select, how='difference')           # difference them both  (listed as inner then outer)
%time LU_frame_expanded = gpd.GeoDataFrame( pd.concat( [Buildings_select, erase_step6], ignore_index=True) )    # Keep the inner one 

LU_frame_expanded.to_file(os.path.join(workspace, 'Finished_dataframetest1.shp'))


Wall time: 11h 56min 39s
Wall time: 0 ns
Wall time: 14h 31min 34s
Wall time: 0 ns
Wall time: 11h 53min 45s
Wall time: 0 ns


In [0]:
# old syntax 
Erasein1 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Buildings')))
Erasein2 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format('Impervious_Surfaces')))

%time erasestep1time = gpd.overlay(Erasein1, Erasein2, how='difference')
erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong1')))
                       
print("unioning back in on copymaster {}".format('1'))
%time erasestep2time = gpd.overlay(Erasein1, Erasein2, how='union')   
erasestep1time.to_file(os.path.join(workspace, 'Buffer_{}.shp'.format('testbong2')))

In [0]:
for idx, lutype in enumerate(type_list):
    print("erasing on copymaster {}".format(lutype))
    Erasein1 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)))
    Erasein2 = gpd.read_file(os.path.join(workspace, 'Buffer_{}.shp'.format(type_list[idx+1])))
    
    %time newlandusefile = gpd.overlay(Erasein1, Erasein2, how='difference')
    newlandusefile.to_file(copy_LU_shp)
       
    print("unioning back in on copymaster {}".format(lutype))
    %time newlandusefile2 = gpd.overlay(Erasein1, Erasein2, how='union')   
    newlandusefile2.to_file(copy_LU_shp)

In [0]:
Process: 
    - project the land use file 
    - define the buffer distances for each land use type
    - for each land use type: 
        - create stand alone shapefile of desired type
        - buffer it by desired amount 
        - 
        
        
        
############## eratly attempts to do an erase with ogr

driver = ogr.GetDriverByName('ESRI Shapefile')
outDriver = ogr.GetDriverByName("ESRI Shapefile")  

inlayer = driver.Open(os.path.join(workspace, 'Buffer_Open_Space.shp'), 0)
eraselayer = driver.Open(os.path.join(workspace, 'Buffer_Buildings.shp'), 0)
outDataSource = outDriver.CreateDataSource(os.path.join(workspace, 'trashfaceman.shp'))


feat1.Erase(eraselayer, outDataSource)

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x000001881365D810> >

In [0]:
good links 
https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
    https://gis.stackexchange.com/questions/163040/erasing-features-using-python-gdal-ogr
    